In [ ]:
from classes.Dataset_Basic import Dataset_Basic
from classes.Process_Stages import Process_Stages

from classes.Dataset_Settings import Dataset_Settings

from run_models.cosine_sililarity.classes.Dataset_Cosine import Dataset_Cosine
from run_models.gensim.classes.Process_Stages_Gensim import Process_Stages_Gensim

from run_models.embed_words.classes.Embed_Words import Embed_Words

from run_models.gensim.services.gensim_services import gensim_download, gensim_save, gensim_load
from word_embedding.models.classes.EmbeddingModel import EmbeddingModel

name_dataset = "conceptnet_cosine_similarity"

datasets = {}
datasets["neural_course"] = Dataset_Cosine(
    df_name="neural_course",
    model_name=name_dataset, # used for dir name inside data_saved

    language="english",

    datasets = {
        "emebed_words_conceptnet": Dataset_Settings(
            df=None,
            may_run_now=False,
            required=True,
            parquet=True
        ),
        name_dataset: Dataset_Settings(
            df=None,
            may_run_now=True,
            required=True,
            parquet=True,
            name_required_dataset="gensim"
        ),
    },

    columns_to_add = {name_dataset: {"cosine_similarity": []}},

)

# get dataset, process dataset, save dataset
datasets["neural_course"].get_dataset()
datasets["neural_course"].process_dataset()
datasets["neural_course"].add_columns()
datasets["neural_course"].save()


# BERT embedding model

In [ ]:
from transformers import BertModel, BertTokenizer
import torch

# Initialize the tokenizer and the model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Your sentence
sentence = "This is a simple sentence."

# Tokenize the sentence and encode it to IDs
input_ids = tokenizer.encode(sentence, add_special_tokens=True)

# Create tensors
input_ids = torch.tensor([input_ids])

# Forward pass: compute the BERT embeddings
with torch.no_grad():
    outputs = model(input_ids)

# Extract the last hidden states (the embeddings)
last_hidden_states = outputs[0]

# # Print the embeddings
# for i, token_str in enumerate(tokenizer.convert_ids_to_tokens(input_ids[0])):
#     print(token_str, last_hidden_states[0, i])



In [ ]:
def tokenize_sentence(sentence):
    # Tokenize the sentence and encode it to IDs
    input_ids = tokenizer.encode(sentence, add_special_tokens=True)

    # Create tensors
    input_ids = torch.tensor([input_ids])

    # Forward pass: compute the BERT embeddings
    with torch.no_grad():
        outputs = model(input_ids)

    # Extract the last hidden states (the embeddings)
    last_hidden_states = outputs[0]

    return last_hidden_states

In [ ]:
# classes
from data.embed_words.BERT_embedding.classes.BERT_Embedding import BERT_Embedding
from classes.Dataset_Settings import Dataset_Settings

BERT_embed = BERT_Embedding(
    df_name="neural_course",
    model_name="BERT", # used for dir name inside data_saved

    language="english",

    datasets = {
        "standardized_splits": Dataset_Settings(
            df=None,
            df_name="splits",
            base_dir="data",

            may_run_now=False,
            required=True,
        ),
        "BERT": Dataset_Settings(
            df=None,
            df_name="BERT",
            base_dir="data/embed_words/data",

            may_run_now=True,
            required=True,
            parquet=True,
            name_required_dataset="standardized_splits",
            force_run=False
        ),
    },

    pre_trained_BERT_model_name="bert-base-uncased"

)

BERT_embed.run_all()


In [ ]:
"Marly is lief"

In [ ]:
# embed a sentence in BERT_tokenized
import pandas as pd

df_BERT = pd.read_parquet("data/embed_words/data/BERT/data/neural_course.parquet")
df_2 = pd.read_csv("data/splits/data/neural_course.csv")
fast_text = pd.read_parquet("data/embed_words/data/fasttext/data/neural_course.parquet")

basic = pd.read_csv("data/basic_processed/data/neural_course.csv")


In [ ]:
from services.string_array import str_to_array
from scipy.spatial.distance import cosine

1 - cosine(str_to_array(df_BERT.iloc[0]["student_answer"])[0], str_to_array(df_BERT.iloc[0]["student_answer"])[0])


In [4]:
import pandas as pd

# create sample dataset
sample = pd.read_csv("data/splits/data/beetle.csv")
sample = sample.sample(100, random_state=42)


In [ ]:
from sklearn.model_selection import train_test_split

# Split the DataFrame into train (70%) and the remaining data (30%)
train, temp_df = train_test_split(sample, test_size=0.3, random_state=42)

# Split the remaining data further into test (20% of the original dataset) and validation (10% of the original dataset) sets
test, validation = train_test_split(temp_df, test_size=1/3, random_state=42)


In [ ]:
validation.iloc[0:].shape[0]


In [ ]:
from services.save import save

save(
    dir="testing/data", 
    file_name="sample", 
    df=sample
)


In [5]:
sample.query("question_id == 8")


,row_id,question,question_id,student_answer,reference_answer,assigned_points,max_points,domain,dataset_name,normalized_points
96,96,Explain why you got a voltage reading of 1.5 f...,8,Terminal 1 is not connected to terminal 1,There is a gap between terminals 1 and 6,0,1,NaN,beetle,0.0


In [1]:
from performance_tracking.classes.Dataset_api import Dataset_api

dataset = Dataset_api(
    dir="testing/data",
    file_name="sample",
    seed=42,
    shots=3 # DO NOT CHANGE UNTILL FINISHED WITH RUN THROUGH ALL DATASETS!
)

dataset.split_datasets()


Found sample with type csv in (testing/data) and converted it to df
   row_id                                           question  question_id  \
0    1488  How does a damaged bulb compare to an open swi...           70   
1    6348                            Explain your reasoning.          127   
2    1321  Explain why you got a voltage reading of 1.5 f...           68   
3    2927  When switch X was closed and switch Y was open...           86   
4    4516      Explain why circuit 2 is not a short circuit.          106   
5    3901          Explain why circuit 5 is a short circuit.           98   
6    3749  What does a voltage reading of 1.5 tell you ab...           96   
7    3090  Explain why you got a voltage reading of 1.5 f...           88   
8    1536  What role does the path play in determining wh...           71   
9    4882  Explain why you got a voltage reading of 1.5 f...          109   

                                      student_answer  \
0                         it

In [ ]:
# dataset["train"].query("question_id == 70")


In [2]:
len(dataset["validation"])


8

In [ ]:
dataset["validation"].head(2)


# Results

In [ ]:
performance = pd.read_csv("performance_tracking/tracking/performance_tracking.csv")
performance = performance.query("dataset_split == 'validation_df'")
performance = performance.query("dataset_name != 'concatenated_datasets'")
performance = performance.query("dataset_name != 'concatenated_domains'")


In [ ]:
performance.head(2)

In [ ]:

grouped_column = "embedding_model_name"

# embedding_model_name classification_model_name
performance_embed_avg = performance.groupby(grouped_column).mean().reset_index()
performance_embed_avg_classification = performance_embed_avg.loc[:, [grouped_column, "accuracy", "precision_weighted", "recall_weighted", "f1_weighted"]]
performance_embed_avg_classification = performance_embed_avg_classification.round(2)

performance_embed_avg_cont = performance_embed_avg.loc[:, [grouped_column, "rmse", "pears_correlation", "p_value"]]
performance_embed_avg_cont = performance_embed_avg_cont.round(2)

performance_embed_std = performance.groupby(grouped_column).std().reset_index()
performance_embed_std_classification = performance_embed_std.loc[:, [grouped_column, "accuracy", "precision_weighted", "recall_weighted", "f1_weighted"]]
performance_embed_std_classification = performance_embed_std_classification.round(2)

performance_embed_std_cont = performance_embed_std.loc[:, [grouped_column, "rmse", "pears_correlation", "p_value"]]
performance_embed_std_cont = performance_embed_std_cont.round(2)


In [ ]:
print(performance_embed_avg_classification.to_latex(index=False))





In [ ]:
print(performance_embed_avg_cont.to_latex(index=False))


In [ ]:
print(performance_embed_std_classification.to_latex(index=False))


In [ ]:
print(performance_embed_std_cont.to_latex(index=False))

In [ ]:
import torch
from transformers import BertModel, BertTokenizer

# Load pre-trained model tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')

# Load pre-trained model
model = BertModel.from_pretrained('bert-large-uncased')

# Sentence for which we want to create embeddings
sentence = "This is a sample sentence."

# Tokenize the sentence and convert to input IDs.
inputs = tokenizer(sentence, return_tensors='pt')

# Run the sentence through the model to get the embeddings.
with torch.no_grad():
    outputs = model(**inputs)

# We get a tuple as output. The first element of the tuple is the embeddings.
embeddings = outputs[0]

print(embeddings)


In [ ]:
len(embeddings[0])

In [ ]:
inputs

In [ ]:
from transformers import BertModel, BertTokenizer

# Load pre-trained model tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load pre-trained model
model = BertModel.from_pretrained('bert-base-uncased')

# A very long sentence
sentence = "This is a very long sentence..." # Assume this is a very long sentence

# Tokenize the sentence and get the tokens
tokens = tokenizer.tokenize(sentence)

# Define the maximum chunk size (max BERT length - 2 for [CLS] and [SEP])
max_chunk_size = 510

# Split the tokens into chunks
chunks = [tokens[i:i + max_chunk_size] for i in range(0, len(tokens), max_chunk_size)]

# For each chunk, add special tokens, convert to input IDs, feed to BERT, and get embeddings
for chunk in chunks:
    # Add special tokens
    chunk = ['[CLS]'] + chunk + ['[SEP]']
    # Convert to input IDs
    inputs = tokenizer.convert_tokens_to_ids(chunk)
    inputs = torch.tensor([inputs])  # Add batch dimension
    # Get embeddings
    with torch.no_grad():
        outputs = model(inputs)
    # The first element of outputs is the embeddings
    embeddings = outputs[0]
    print(embeddings)


In [ ]:
from performance_tracking.classes.Performance_Row import Performance_Row


In [ ]:
row = Performance_Row(
    # id what is being tracked
    dataset_name="test",
    embedding_seperated=self.measurement_settings.embedding_seperated,
    embedding_model_name = self.measurement_settings.embedding_model_name,
    sentence_embedding_method = self.measurement_settings.sentence_embedding_method,
    feature_engenearing_method = self.measurement_settings.feature_engenearing_method,
    grading_model = self.measurement_settings.grading_model,
    seed_data_split = self.measurement_settings.seed_data_split,
    shots=self.shots,
    epochs=self.epochs,

    dataset_split=dataset_split,

    # duplicates handeling
    settings_performance_tracking=self.measurement_settings.settings_performance_tracking
)


